# Regression Multi Variable 

In [ ]:
#Boston Housing Dataset: Load the boston dataset.
from sklearn.datasets import load_boston
boston = load_boston()

import pandas as pd

df_boston = pd.DataFrame(boston.data,columns=boston.feature_names)
df_boston['target'] = boston.target
df_boston.head()

In [ ]:
df_boston.shape

In [ ]:
# visualize the relationship between the features and the response using scatterplots
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#sns.pairplot(df_boston, x_vars=boston.feature_names, y_vars='target')


fig, axes = plt.subplots(3, 5,figsize=[15,8],constrained_layout=True)
axes = axes.flatten()
i=0
for x in df_boston.columns[:-1]:
    plt.sca(axes[i]) # set the current Axes
    plt.scatter(df_boston[x],df_boston.target)
    plt.xlabel(x)
    plt.ylabel("target")
    i+=1
    
plt.show()


In [ ]:
# call regplot on each axes
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True,figsize=[8,4])
sns.scatterplot(x=df_boston['RM'], y=df_boston.target, ax=ax1)
sns.scatterplot(x=df_boston['LSTAT'], y=df_boston.target, ax=ax2)


In [ ]:
import numpy as np 
log_LSTAT=np.log(df_boston['LSTAT'])

log_CRIM=np.log(df_boston['CRIM'])

# call regplot on each axes
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True,figsize=[8,4])
sns.scatterplot(x=df_boston['LSTAT'], y=df_boston.target, ax=ax1)
sns.scatterplot(x=log_LSTAT, y=df_boston.target, ax=ax2)
#sns.scatterplot(x=log_CRIM, y=df_boston.target, ax=ax1)                          

In [ ]:
sns.histplot(data=df_boston, x='target')


In [ ]:
#We can compute the correlation 
df_boston.corr().round(2)

In [ ]:
#We can visualise the correlation using a heatmap in Seaborn

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.figure(figsize = (10,10))
sns.heatmap(data=df_boston.corr().round(2), cmap='coolwarm', linewidths=.5, annot=True, annot_kws={"size":12})
plt.show()

###  Brief discussion on correlation

In [ ]:
import pandas as pd
df_q = pd.read_csv("quadratic.csv") 

df_q.columns

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(df_q["x"], df_q["y"], 'o')

In [ ]:
df_q.corr()

In [ ]:
import math 
df_q["x2"]=df_q["x"].apply(lambda x: -(x-0.5)**2)

In [ ]:
df_q

In [ ]:
df_q.corr()

In [ ]:
from sklearn.linear_model import LinearRegression

simple_regr=LinearRegression()
simple_regr.fit(df_q[["x","x2"]], df_q["y"])

y_pred=simple_regr.predict(df_q[["x","x2"]])


In [ ]:
plt.plot(df_q["y"],y_pred,'o')

In [ ]:
from sklearn import metrics

print("r2: ",metrics.r2_score(df_q["y"], y_pred ))

## Select data

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Separate features and target variables
X = df_boston.iloc[:,:-1] #if I want to use all variables
y = df_boston.iloc[:,-1]

#choose your approach:
#X = df_boston.iloc[:,[4,5,10,11,12]] #if I want to use only some variable
#X = X.drop(['INDUS','CHAS','AGE','B'], axis=1) #if I want to drop some columns
#X = X[['RM','LSTAT']]#if I want to select some columns
X["LSTAT2"] = np.log(X.LSTAT) #if I want to log transform the LSTAT variable 
#X["CRIM2"] = np.log(X.CRIM) #if I want to log transform the LSTAT variable 

columns = X.columns #column names

In [ ]:
X=X[['RM','LSTAT2']]

# Filter the unusual observation
#X=X[y<50]
#y=y[y<50]

In [ ]:
y.head(3)

In [ ]:
X.head()

In [ ]:
#Scale and select Train/Test
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=False).fit(X)
scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.3,
                                                    random_state=123)


In [ ]:
#DEFINE YOUR REGRESSOR and THE PARAMETERS GRID
from sklearn.linear_model import LinearRegression
import numpy as np

regressor = LinearRegression() #(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
parameters = {}

#DEFINE YOUR GRIDSEARCH 
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(regressor, parameters,cv=3) #with no params it reduces to a CV

gs = gs.fit(X_train,y_train)

#summarize the results of your GRIDSEARCH
print('***GRIDSEARCH RESULTS***')
print("Best score: %f using %s" % (gs.best_score_, gs.best_params_))
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
params = gs.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#test on hold-out

#gs.score(X_train, y_train)
gs.score(X_test, y_test)

In [ ]:
plt.scatter(y_test, gs.predict(X_test))
plt.xlabel("Prices: $y_i$")
plt.ylabel("Predicted prices: $\hat{y}_i$")
plt.title("Prices vs Predicted prices: $y_i$ vs $\hat{y}_i$")

In [ ]:
#Independent term in the linear model.
print('Intercept: ', gs.best_estimator_.intercept_)

gs.best_estimator_.coef_

#import pandas as pd
#pd.DataFrame(list(zip(columns,gs.best_estimator_.coef_)), columns = ['features','estimatedCoefficients'])

In [ ]:
from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error

print("MSE train: ", mean_squared_error(y_train, gs.predict(X_train)))
print("MSE test: ", mean_squared_error(y_test, gs.predict(X_test)))

# computing MAE, MSE, RMSE, r²
 - Mean Absolute Error (MAE): $$\frac 1n\sum_{i=1}^n|y_i-\hat{y}_i|$$
 - Mean Squared Error  (MSE): $$\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2$$
 - Root Mean Squared Error (RMSE) : $$\sqrt{\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2}$$

In [ ]:
from sklearn import metrics

print("MAE train: ", metrics.mean_absolute_error(y_train, gs.predict(X_train))) 
print("MSE train: ",metrics.mean_squared_error(y_train, gs.predict(X_train)))
print("RMSE train: ",np.sqrt(metrics.mean_squared_error(y_train, gs.predict(X_train))))
print("r2: ",metrics.r2_score(y_train, gs.predict(X_train)))

print("MAE test: ", metrics.mean_absolute_error(y_test, gs.predict(X_test))) 
print("MSE test: ",metrics.mean_squared_error(y_test, gs.predict(X_test)))
print("RMSE test: ",np.sqrt(metrics.mean_squared_error(y_test, gs.predict(X_test))))
print("r2: ",metrics.r2_score(y_test, gs.predict(X_test)))

In [ ]:
error_train=gs.predict(X_train)-y_train
error_test=gs.predict(X_test)-y_test

error_train.describe()

In [ ]:
plt.scatter(gs.predict(X_train),error_train, c="b", label="training data")
plt.scatter(gs.predict(X_test),error_test, c="g", label="test data")
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.legend(loc="upper left")
plt.hlines(y=0, xmin=-10, xmax=50, color="r")
plt.xlim([-10,50])
plt.show()

In [ ]:
nb_error_train = np.array(error_train).flatten()

error_train = np.array(error_train).reshape(-1,1)
scaled_error_train= StandardScaler(copy=False).fit(error_train).transform(error_train).flatten()


In [ ]:
import numpy as np
import scipy 
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot_2samples
from matplotlib import pyplot as plt

# We test a exponential distribution
dist = getattr(scipy.stats, 'norm')
param = dist.fit(nb_error_train)

err_mean=param[-2]
err_std=param[-1]

# We generate a sample of size  len(mr_scaled) of data distributed according to distribution dist
# The function rvs generates a sample with distribution dist with mean loc and std scale
test_dist = dist.rvs(*param[0:-2],loc=param[-2], scale=param[-1],size = len(nb_error_train))

# qq-plot using statsmodels
qqplot_2samples(test_dist,np.array(nb_error_train).flatten(), line='45')
plt.show()

# We create the percentiles for both distributions
test_dist.sort()
percs = np.linspace(0,100,21)
q_a = np.percentile(nb_error_train, percs)
q_b = np.percentile(test_dist, percs)

# and generate the QQ-plot 
plt.plot(q_a,q_b, ls="", marker="o")
plt.title("QQ plot")
x = np.linspace(np.min((q_a.min(),q_b.min())), np.max((q_a.max(),q_b.max())))
plt.plot(x,x, color="k", ls="--")
plt.show()


# plot the distribution and compare with a normal

ax = sns.histplot(nb_error_train, stat='density')

# calculate the pdf
x0, x1 = ax.get_xlim()  # extract the endpoints for the x-axis
x_pdf = np.linspace(x0, x1, 100)
y_pdf = scipy.stats.norm.pdf(x_pdf, loc=err_mean, scale=err_std)

ax.plot(x_pdf, y_pdf, 'r', lw=2, label='normal')                                                   
ax.legend() 

#plt.hist(nb_error_train,alpha=.3, density=True,bins='auto')
#plt.hist(test_dist,alpha=.3, density=True,bins='auto')
#plt.show()

In [ ]:
# Kolmogorov-Smirnov Test
from scipy import stats
print(stats.kstest(scaled_error_train, "norm"))
print(stats.kstest(nb_error_train, test_dist))
# normality tests use a (0,1) normal distribution 
# D'agostino normality test
print(stats.normaltest(scaled_error_train))
# Shapiro test of normality
print(stats.shapiro(scaled_error_train))

## Test parameters (statsmodels)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

X_train = sm.add_constant(X_train)
#If we want to add a constant to our model 
est = sm.OLS(y_train, X_train)
est_fit = est.fit()
est_fit.params

In [ ]:
print(est_fit.summary())